## 에이전트를 도구로 만들고 여러개의 에이전트를 도구로 엮는 방법을 보여준다.

In [1]:
import asyncio
from agent_framework.azure import AzureOpenAIChatClient
from azure.identity import AzureCliCredential
from dotenv import load_dotenv

print("라이브러리 임포트 완료!")

라이브러리 임포트 완료!


In [2]:
# 환경 변수 로드
load_dotenv()
print("환경 변수 로드 완료!")

# Azure CLI 인증 준비
credential = AzureCliCredential()
print("Azure CLI 인증 설정 완료!")

환경 변수 로드 완료!
Azure CLI 인증 설정 완료!


## Creating and using an agent as a function tool
https://learn.microsoft.com/en-us/agent-framework/tutorials/agents/agent-as-function-tool?pivots=programming-language-python

In [3]:
from typing import Annotated
from pydantic import Field

def get_weather(
    location: Annotated[str, Field(description="The location to get the weather for.")],
) -> str:
    """Get the weather for a given location."""
    return f"The weather in {location} is cloudy with a high of 15°C."

In [5]:
# Function Tool을 사용하도록 WeatherAgent 이름으로 ChatAgent를 생성한다. 

from agent_framework.azure import AzureOpenAIChatClient
from azure.identity import AzureCliCredential

weather_agent = AzureOpenAIChatClient(credential=AzureCliCredential()).create_agent(
    name="WeatherAgent",
    description="An agent that answers questions about the weather.",
    instructions="You answer questions about the weather.",
    tools=get_weather
)

In [6]:
# main agent를 생성하면서, weather_agent를 function tool로 추가한다.
main_agent = AzureOpenAIChatClient(credential=AzureCliCredential()).create_agent(
    instructions="You are a helpful assistant who responds in French.",
    tools=weather_agent.as_tool()
)

In [7]:
result = await main_agent.run("What is the weather like in Amsterdam?")
print(result.text)

Actuellement à Amsterdam le ciel est couvert et la journée devrait atteindre environ 15 °C. Voulez-vous la température actuelle exacte, le vent, la probabilité de précipitations ou une prévision horaire ?


In [9]:
# 또는 Agent를 as_tool 함수를 통하여 tool로 변환한다. 
weather_tool = weather_agent.as_tool(
    name="WeatherLookup",
    description="Look up weather information for any location",
    arg_name="query", 
    arg_description="The weather query or location"
)

main_agent = AzureOpenAIChatClient(credential=AzureCliCredential()).create_agent(
    instructions="You are a helpful assistant who responds in French.",
    tools=weather_tool
)
result = await main_agent.run("What is the weather like in Amsterdam?")
print(result.text)

À Amsterdam, c'est nuageux et la maximale prévue est de 15 °C. Prévoyez une veste légère. Voulez-vous les détails heure par heure, la probabilité de précipitations ou une prévision sur plusieurs jours ?
